In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import pandas.testing as tm
import os
import sklearn_pandas
from sklearn.model_selection import train_test_split

In [ ]:
# processing code from set A and D patients:
data = pd.read_csv('F/F010.txt')
#_______________________________
# the below here is for generalization and to fit the model:
datas = [pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
folders = ['Z','F','N','O','S']
for folder in folders:
    i = folders.index(folder)
    temp_data = pd.DataFrame()
    for filename in sorted(os.listdir(folder)):
        temp = pd.read_csv(folder + "/" + filename, sep = "\n", header=None).to_numpy().flatten()
        temp_data[filename] = temp
    datas[i] = temp_data

trainZ, testZ = train_test_split(datas[0].T, test_size=0.1, random_state=42, shuffle=False)
trainF, testF = train_test_split(datas[1].T, test_size=0.1, random_state=42, shuffle=False)
trainN, testN = train_test_split(datas[2].T, test_size=0.1, random_state=42, shuffle=False)
trainO, testO = train_test_split(datas[3].T, test_size=0.1, random_state=42, shuffle=False)
trainS, testS = train_test_split(datas[4].T, test_size=0.1, random_state=42, shuffle=False)

print(trainZ.shape)
print(testZ.shape)
print(trainF.shape)
print(testF.shape)
print(trainS.T)

#pd.DataFrame(datas[4]).to_csv("./Sset.csv")

(90, 4097)
(10, 4097)
(90, 4097)
(10, 4097)
      S001.txt  S002.txt  S003.txt  ...  S088.txt  S089.txt  S090.txt
0          100       340      -310  ...        80       151       134
1          124       353        93  ...        86        25       390
2          153       400       494  ...        91       -82       446
3          185       470       789  ...       100      -134       329
4          210       538       798  ...       107      -102        49
...        ...       ...       ...  ...       ...       ...       ...
4092         7       829      -735  ...        43         5      -104
4093        26       722      -847  ...        42        46      -189
4094        57       512      -721  ...        54        84      -153
4095        95       130      -491  ...        78       119       -62
4096       462       196       910  ...        89      -226        72

[4097 rows x 90 columns]


In [ ]:
# Data augumentation schema 1
#______________________________________________________
#Train set: 90 * 57 = 5130 instances
# Choosing a window size of 512 and a stride of 64 (12.5% of 512 with an overlap of 87.5%), each signal of length 4097 in the training set is divided into
# 57 sub-signals, each of which is treated as an independent signal instance 𝑆𝑡𝑟. 4096
#__________________________________________________________________________________
def trainAug1(trainSet):
  Augument1 = []
  for j in range(90):
    for i in range(57):
      cut1 = trainSet.iloc[:,j][i*64:512+64*i].values.flatten()
      Augument1.append(cut1)
  return pd.DataFrame(tf.keras.utils.normalize(np.array(Augument1).T))

trainInstancesZ1 = trainAug1(trainZ.T)
trainInstancesF1 = trainAug1(trainF.T)
trainInstancesN1 = trainAug1(trainN.T)
trainInstancesO1 = trainAug1(trainO.T)
trainInstancesS1 = trainAug1(trainS.T)
Xtrain = pd.concat([trainInstancesZ1, trainInstancesO1, trainInstancesF1, trainInstancesN1,trainInstancesS1], axis=1, sort=False, ignore_index=True)

labelZ1 = pd.DataFrame(np.zeros((5130,), dtype=int)).T
labelF1 = pd.DataFrame(np.ones((5130,), dtype=int)).T
labelN1 = pd.DataFrame(np.ones((5130,), dtype=int)).T
labelO1 = pd.DataFrame(np.zeros((5130,), dtype=int)).T
labelS1 = pd.DataFrame(np.ones((5130,), dtype=int)).T

Ytrain = pd.concat([labelZ1, labelO1, labelF1, labelN1,labelS1], axis=1, sort=False, ignore_index=True)
print(Xtrain.shape)
print(Xtrain)
print(Ytrain)
#__________________________________________________________________________________
#Test set: 10 * 4 * 3 = 120
def testAug1(testSet):
  Divide1 = []
  for m in range(10):
    for k in range(4):
      divide1 = testSet.iloc[:,m][k*1024:1024+k*1024].values.flatten()
      Divide1.append(divide1)
  divX1 = pd.DataFrame(np.array(Divide1))
  CUT1 = []
  for r in Divide1:
    for p in range(3):
      chop1 = r[p*256:512+256*p]
      CUT1.append(chop1)
  return pd.DataFrame(tf.keras.utils.normalize(np.array(CUT1).T))

testInstancesZ1 = testAug1(testZ.T)
testInstancesF1 = testAug1(testF.T)
testInstancesN1 = testAug1(testN.T)
testInstancesO1 = testAug1(testO.T)
testInstancesS1 = testAug1(testS.T)

Xtest = pd.concat([testInstancesZ1, testInstancesO1, testInstancesF1, testInstancesN1,testInstancesS1], axis=1, sort=False, ignore_index=True)

labelZ1 = pd.DataFrame(np.zeros((120,), dtype=int)).T
labelF1 = pd.DataFrame(np.ones((120,), dtype=int)).T
labelN1 = pd.DataFrame(np.ones((120,), dtype=int)).T
labelO1 = pd.DataFrame(np.zeros((120,), dtype=int)).T
labelS1 = pd.DataFrame(np.ones((120,), dtype=int)).T

Ytest = pd.concat([labelZ1, labelO1, labelF1, labelN1,labelS1], axis=1, sort=False, ignore_index=True)


print(Xtest)
print(Ytest)

(512, 25650)
        0         1         2      ...     25647     25648     25649
0    0.003459 -0.003748 -0.005766  ... -0.000831 -0.006526 -0.005487
1    0.006296 -0.002003 -0.008013  ...  0.000125 -0.006748 -0.010788
2    0.009924 -0.000284 -0.011909  ... -0.000369 -0.005776 -0.012863
3    0.012822  0.000000 -0.011682  ... -0.001910 -0.007071 -0.014996
4    0.019750 -0.003149 -0.005725  ... -0.001729 -0.005023 -0.014492
..        ...       ...       ...  ...       ...       ...       ...
507  0.005151  0.012878 -0.050941  ...  0.002079  0.016465  0.003777
508  0.007725  0.013448 -0.044635  ...  0.000976  0.009716 -0.004412
509  0.009506  0.011810 -0.038887  ...  0.000628  0.011687 -0.007917
510  0.008377  0.008666 -0.031486  ...  0.001484  0.005522 -0.006305
511  0.004619  0.004042 -0.022518  ...  0.003095 -0.008170 -0.002558

[512 rows x 25650 columns]
   0      1      2      3      4      ...  25645  25646  25647  25648  25649
0      0      0      0      0      0  ...      1      

In [ ]:
#Data augumentation schema 2
#[i*128:512+128*i] range(29)
#[k*1024:1024+k*1024] range(4)
#[p*128:512+128*p] range(5)
def trainAug2(trainSet):
  Augument2 = []
  for j in range(90):
    for i in range(29):
      cut2 = trainSet.iloc[:,j][i*128:512+128*i].values.flatten()
      Augument2.append(cut2)
  return pd.DataFrame(tf.keras.utils.normalize(np.array(Augument2).T))

trainInstances2 = trainAug2(trainZ.T)
print(trainInstances2)
#__________________________________________________________________________________
#Test set: 10 * 4 * 3 = 120
def testAug2(testSet):
  Divide2 = []
  for m in range(10):
    for k in range(4):
      divide2 = testSet.iloc[:,m][k*1024:1024+k*1024].values.flatten()
      Divide2.append(divide2)
  divX2 = pd.DataFrame(np.array(Divide2))
  CUT2 = []
  for r in Divide2:
    for p in range(5):
      chop2 = r[p*128:512+128*p]
      CUT2.append(chop2)
  return pd.DataFrame(tf.keras.utils.normalize(np.array(CUT2).T))

testInstances2 = testAug2(testZ.T)
print(testInstances2)

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
#Adam optimizer
AdamConf = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')

In [ ]:
def P_1D_CNN(opt):
  model = models.Sequential()
  #layer 1:
  model.add(layers.Conv1D(5, 24, strides=3, activation=None, input_shape=(512,1)))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  #layer 2:
  model.add(layers.Conv1D(3, 12, strides=2, activation=None))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  #layer 3:
  model.add(layers.Conv1D(3, 8, strides=2, activation=None))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  #FC1
  model.add(layers.Flatten())
  model.add(layers.Dense(40))
  model.add(layers.ReLU())
  #DO
  model.add(layers.Dropout(0.5))
  #FC2
  model.add(layers.Dense(1, activation='sigmoid'))
  #Compiling
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
Xtrain = np.expand_dims(Xtrain.T, 2)
Xtest = np.expand_dims(Xtest.T, 2)
Ytrain = np.expand_dims(Ytrain.T, 2)
Ytest = np.expand_dims(Ytest.T, 2)

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)
print(Ytrain.shape)
print(Ytest.shape)

(25650, 512, 1)
(600, 512, 1)
(25650, 1, 1)
(600, 1, 1)


In [ ]:
# Fit first to see what happen: iter = epochs*len(L)/batch
oneAtaTime = P_1D_CNN()

# Model chekpoint:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./", monitor='val_accuracy', verbose=False, save_best_only=True,
    save_weights_only=False, mode='max', save_freq='epoch'
)

# Training loop:
history = oneAtaTime.fit(Xtrain, Ytrain, batch_size=32, epochs=200, callbacks=[checkpoint], validation_data=(Xtest,Ytest))


array([[206,  34],
       [  8, 352]])

In [ ]:
# define the grid search parameters
# validation_data=(Xtest,Ytest)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
X = np.concatenate((Xtrain, Xtest), axis=0)
Y = np.concatenate((Ytrain, Ytest), axis=0)


for train, test in kfold.split(X, Y):

  batch_size = [32]
  epochs = [50]
  learn_rate = [0.01]
  for b in batch_size:
    for e in epochs:
      for l in learn_rate:
        print('THE FOLLOWING IS FOR LEARNING RATE ' + str(l) +', BATCH SIZE ' + str(b) +', AND EPOCHS ' + str(e))
        AdamConf = tf.keras.optimizers.Adam(learning_rate=l, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
        checkpoint = tf.keras.callbacks.ModelCheckpoint("./", monitor='val_accuracy', verbose=False, save_best_only=True,save_weights_only=False, mode='max', save_freq='epoch') 
        model = P_1D_CNN(AdamConf)
        history = model.fit(X[train], Y[train], batch_size=b, epochs=e, callbacks=[checkpoint], validation_split=0.25)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        # Generate generalization metrics
        scores = model.evaluate(X[test], Y[test], verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])

        # Increase fold number
        fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')




THE FOLLOWING IS FOR LEARNING RATE 0.01, BATCH SIZE 32, AND EPOCHS 50
Epoch 1/50
554/554 [==============================] - 8s 14ms/step - loss: 0.1867 - accuracy: 0.9279 - val_loss: 0.0602 - val_accuracy: 0.9826
Epoch 2/50
554/554 [==============================] - 5s 9ms/step - loss: 0.1157 - accuracy: 0.9616 - val_loss: 0.4644 - val_accuracy: 0.8915
Epoch 3/50
554/554 [==============================] - 5s 9ms/step - loss: 0.0920 - accuracy: 0.9686 - val_loss: 1.8178 - val_accuracy: 0.9636
Epoch 4/50
554/554 [==============================] - 5s 9ms/step - loss: 0.1007 - accuracy: 0.9685 - val_loss: 1.1629 - val_accuracy: 0.9636
Epoch 5/50
554/554 [==============================] - 5s 9ms/step - loss: 0.0842 - accuracy: 0.9708 - val_loss: 1.1108 - val_accuracy: 0.9638
Epoch 6/50
554/554 [==============================] - 5s 9ms/step - loss: 0.0894 - accuracy: 0.9692 - val_loss: 2.8333 - val_accuracy: 0.2600
Epoch 7/50
554/554 [==============================] - 5s 9ms/step - loss: 0.0

In [ ]:
batch_size = [32,64]
epochs = [20,50]
learn_rate = [0.001,0.01]
for b in batch_size:
  for e in epochs:
    for l in learn_rate:
      print('THE FOLLOWING IS FOR LEARNING RATE ' + str(l) +', BATCH SIZE ' + str(b) +', AND EPOCHS ' + str(e))
      AdamConf = tf.keras.optimizers.Adam(learning_rate=l, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
      checkpoint = tf.keras.callbacks.ModelCheckpoint("./", monitor='val_accuracy', verbose=False, save_best_only=True,save_weights_only=False, mode='max', save_freq='epoch') 
      model = P_1D_CNN(AdamConf)
      history = model.fit(Xtrain, Ytrain, batch_size=b, epochs=e, callbacks=[checkpoint], validation_data = (Xtest,Ytest) )
      
     

THE FOLLOWING IS FOR LEARNING RATE 0.001, BATCH SIZE 32, AND EPOCHS 20
Epoch 1/20
802/802 [==============================] - 9s 11ms/step - loss: 0.3455 - accuracy: 0.8514 - val_loss: 0.5498 - val_accuracy: 0.8833
Epoch 2/20
802/802 [==============================] - 9s 12ms/step - loss: 0.2244 - accuracy: 0.9179 - val_loss: 0.7857 - val_accuracy: 0.9533
Epoch 3/20
802/802 [==============================] - 6s 8ms/step - loss: 0.2052 - accuracy: 0.9237 - val_loss: 1.2726 - val_accuracy: 0.8867
Epoch 4/20
802/802 [==============================] - 9s 11ms/step - loss: 0.1926 - accuracy: 0.9256 - val_loss: 0.3720 - val_accuracy: 0.9650
Epoch 5/20
802/802 [==============================] - 7s 8ms/step - loss: 0.1842 - accuracy: 0.9281 - val_loss: 1.4377 - val_accuracy: 0.8917
Epoch 6/20
802/802 [==============================] - 9s 12ms/step - loss: 0.1783 - accuracy: 0.9290 - val_loss: 0.2468 - val_accuracy: 0.9733
Epoch 7/20
802/802 [==============================] - 7s 8ms/step - loss:

After training we will creating n P-1D-CNN model to predict which signal is this and later will use majority voting to finalize the decision. In schema 1, there would be 3 P1DCNN model since the test signal is divided into 4 independent signal and then each divide into 3 independent ones, while schema 2 is 5

In [ ]:
oneAtaTime = tf.keras.models.load_model("./")
Ypred = ((oneAtaTime.predict(Xtest) > 0.5).astype(int)).ravel()
confusion_matrix(Ytest.astype(int).ravel(), Ypred)

array([[ 22, 218],
       [  0, 360]])